In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from yellowbrick.target import FeatureCorrelation
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer

In [75]:
df = pd.read_csv('clean_data.csv')

In [76]:
df.genre.value_counts()

Indie              9305
Pop                9052
Hip-Hop            9003
Rap                8966
Electronic         8897
                   ... 
wisconsin indie       1
country dawn          1
indian edm            1
twee pop              1
chinese hip hop       1
Name: genre, Length: 1369, dtype: int64

In [77]:
genre = (df["genre"].str.strip()).str.lower()

def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ",1)
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)

len(genre_m1.unique())

456

In [78]:
unique = genre_m1.unique()
to_remove = [] 

# genres that have number of songs < 500 will be removed
for genre in unique:
    if genre_m1.value_counts()[genre] < 500:
        to_remove += [genre]
len(to_remove)

426

In [79]:
df['genre'] = genre_m1
df

,name,artist,id,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_min,year
0,6 8,Gabriel Garzón-Montano,0uGEDEO3GjRcnZLFvUxKuA,0,r&b,0.787,0.0577,0,-13.125,1,0.0755,0.88300,0.001330,0.0714,0.157,115.314,4.98,2014
1,To Love Somebody,Bee Gees,3rGFsUgyH9GbbYJT4iwbhJ,0,disco,0.423,0.3840,9,-12.503,1,0.0315,0.72900,0.000004,0.2700,0.521,90.430,3.03,1967
2,"Stayin' Alive - From ""Saturday Night Fever"" So...",Bee Gees,6pyzBTIjcO2AMkKE6x2Otb,0,disco,0.703,0.8260,10,-7.179,0,0.0341,0.03220,0.006290,0.1790,0.945,103.564,4.76,1979
3,"Jive Talkin' - From ""Saturday Night Fever"" Sou...",Bee Gees,4t0Pj3iBnSCZv5pDEPNmzG,64,disco,0.811,0.5450,7,-12.969,0,0.0579,0.15500,0.085300,0.1560,0.734,105.911,3.70,1975
4,"Emotion - New ""Edit"" Version",Bee Gees,2eFM2ufqYVEs0oUA7O5U0O,0,disco,0.689,0.6400,4,-6.991,0,0.0459,0.10700,0.000019,0.1920,0.637,91.834,3.64,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250570,Wait Up,Charlotte Lawrence,6VM9H9tmexHFNLePOXOVPX,51,indie,0.631,0.7050,0,-2.941,0,0.0344,0.22300,0.000032,0.1280,0.382,101.075,3.08,2018
250571,Don't Phunk With My Heart,The Black Eyed Peas,4MtDnnAJ1AtWit10FRmZdW,54,dance,0.691,0.9290,8,-2.730,0,0.0645,0.00877,0.000000,0.4720,0.574,130.899,4.00,2005
250572,Don't Phunk With My Heart,The Black Eyed Peas,4MtDnnAJ1AtWit10FRmZdW,54,rap,0.691,0.9290,8,-2.730,0,0.0645,0.00877,0.000000,0.4720,0.574,130.899,4.00,2005
250573,Don't Phunk With My Heart,The Black Eyed Peas,4MtDnnAJ1AtWit10FRmZdW,54,pop,0.691,0.9290,8,-2.730,0,0.0645,0.00877,0.000000,0.4720,0.574,130.899,4.00,2005


In [80]:
df.set_index(["genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)

In [81]:
df['genre'].value_counts()

pop            15923
music          13937
rock           11770
rap            10690
indie          10501
r&b             9742
country         9631
dance           9070
electronic      9033
hip-hop         9003
soundtrack      8924
comedy          8884
alternative     8758
folk            8723
soul            8593
reggaeton       8501
world           8455
classical       8228
reggae          7949
anime           7854
jazz            7685
ska             7594
blues           7219
opera           7135
movie           6953
hop             3265
metal           1248
edm              629
ccm              595
standards        537
Name: genre, dtype: int64

In [89]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_min','key','mode','year','popularity']

ros = RandomOverSampler()

X = df[feature_names]
y = df['genre']

X_ros, y_ros = ros.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, random_state=42, test_size=0.2)

knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=1))
rf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=10, max_depth=8, min_samples_leaf=8))
nb = make_pipeline(StandardScaler(), GaussianNB())

models = [['KNN', knn], ['Random Forest', rf], ['Naive Bayes', nb]]

for name, model in models:
    model.fit(X_train, y_train)
    print('%s: %f' % (name,model.score(X_test, y_test)))


KNN: 0.695409
Random Forest: 0.373621
Naive Bayes: 0.292742
